In [8]:
import pandas as pd
import torch
from tqdm import tqdm  # For progress tracking
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM


file_path = "/kaggle/input/intentconan-test/test.csv"
df = pd.read_csv(file_path)
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,hatespeechTarget,powerDynamics,prompt_offensiveness,prompt_target_group,prompt_speaker_intent,prompt_power_dynamics,prompt_implication,prompt_emotional_reaction,prompt_cognitive_reaction,prompt_cs_generation
0,20,"Women are basically childlike, they remain thi...",Informative,The characteristics you are referring to are a...,3.0,4.0,2.0,3.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
1,22,"Women are basically childlike, they remain thi...",Denouncing,Nobody apart from people who spread misinforma...,2.0,3.0,3.0,2.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
2,23,"Women are basically childlike, they remain thi...",Positive,It is a question of survival and happiness to ...,2.0,2.0,2.0,2.0,NaN,Human,...,women,gender,Analyze the offensiveness of the statement: Wo...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
3,48,Minorities vote to strip wealth and power from...,Informative,This is incorrect. jews could never be able to...,3.0,4.0,3.0,3.0,NaN,Human,...,jews,religion,Analyze the offensiveness of the statement: Mi...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
4,49,Minorities vote to strip wealth and power from...,Questioning,Where is your evidence?,1.0,1.0,3.0,1.0,NaN,Human,...,jews,religion,Analyze the offensiveness of the statement: Mi...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print( "Using device:", device)


Using device: cuda


In [10]:
%pip install together 

Note: you may need to restart the kernel to use updated packages.


In [11]:
from together import Together
from tqdm import tqdm
import pandas as pd

client = Together(api_key="8359ee49c11106be833a5fc2d7ffc654b7c83aaa8f080da0234a210cae0d57eb")

FEW_SHOT_EXAMPLES = [
    {"hate_speech": "Maybe the UN could talk to those asian and african nations responsible for 90+ of the pollution in the oceans instead of insisting on this bullshit about climate change.",
     "intent": "Positive",
     "counterspeech": "You're right, ocean pollution is one of the main environmental problems we face, however, climate change is a global problem, and everyone should help to combat it, no matter where in the world you are from!"},
    {"hate_speech": "Jews cannot be patriots, since their allegiance will always be to the state of Israel.",
     "intent": "Informative",
     "counterspeech": "You can have parents and grandparents born elsewhere, and still be a patriot for the country you were born in."},
    {"hate_speech": "The British people should use those houses, not immigrants.",
     "intent": "Denouncing",
     "counterspeech": "It is because of people like you that migrants do not feel safe in a place that they should call home. Shame on you."}
]

def generate_counterspeech_togetherai(hate_speech, intent):
    """Generates counterspeech using LLaMA 3.3 70B via TogetherAI API with few-shot examples."""
    examples_text = "\n\n".join([
        f"Hate Speech: {ex['hate_speech']}\nIntent: {ex['intent']}\nCounterspeech: {ex['counterspeech']}"
        for ex in FEW_SHOT_EXAMPLES if ex['intent'] == intent
    ])
    
    prompt = f"""### Counterspeech Generation Task

You are an expert in counterspeech. Generate a **{intent}** counterspeech response to the following hate speech:

### Few-Shot Examples
{examples_text}

### Hate Speech Instance
{hate_speech}

---

### Expected Response
Your response should be a **{intent}** style counterspeech that directly addresses the hate speech instance.
"""
    
    try:
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=250,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1,
            stop=["\n\n", "<|eot_id|>", "<|eom_id|>"],
            stream=False
        )
        
        return response.choices[0].message.content if response.choices else "No response generated."
    except Exception as e:
        print("Error with TogetherAI API:", e)
        return "Error"

# Assuming df is already loaded with hate speech data
df["Llama3_counterspeech"] = ""

print("\nGenerating counterspeech using TogetherAI LLaMA 3.3 70B with few-shot examples...")

for i in tqdm(range(len(df)), desc="Processing Hate Speech"):
    counterspeech = generate_counterspeech_togetherai(
        df.at[i, "hatespeech"], df.at[i, "csType"]
    )
    df.at[i, "Llama3_counterspeech"] = counterspeech
    
    # Save progress every 100 rows to avoid data loss in case of an interruption
    if i % 100 == 0:
        df.to_csv("llama_fs.csv", index=False)
        print(f"Progress saved at row {i}...")

# Final save after processing all rows
df.to_csv("llama_fs_csv.csv", index=False)
print("\nFinal results saved to llama_fs.csv")



Generating counterspeech using TogetherAI LLaMA 3.3 70B with few-shot examples...


Processing Hate Speech:   0%|          | 1/2971 [00:03<3:06:50,  3.77s/it]

Progress saved at row 0...


Processing Hate Speech:   3%|▎         | 101/2971 [13:20<5:41:02,  7.13s/it]

Progress saved at row 100...


...

Progress saved at row 2800...


Processing Hate Speech:  98%|█████████▊| 2901/2971 [6:20:21<07:05,  6.08s/it]

Progress saved at row 2900...


Processing Hate Speech: 100%|██████████| 2971/2971 [6:29:20<00:00,  7.86s/it]


Final results saved to llama_fs.csv
